# Generating Image Embeddings

Multimodal embeddings APIs of Azure AI Vision let you search for images using text. They turn images and text into coordinates in a multi-dimensional space, matching them based on how closely they relate in meaning. This means you can find images without needing tags or metadata, often getting better search results.

To better understand how embeddings work, you can watch my video that explains this in this LinkedIn Video: 
https://www.linkedin.com/learning/building-rag-solutions-with-azure-ai-foundry-formerly-azure-ai-studio/vector-embeddings-how-words-connect-to-each-other

Further Reading:

https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/concept-image-retrieval

https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/how-to/image-retrieval?tabs=python

## Load Azure Configuration

In [ ]:
import os

azure_computer_vision_endpoint = os.environ["AZURE_COMPUTER_VISION_ENDPOINT"]
azure_computer_vision_key = os.environ["AZURE_COMPUTER_VISION_KEY"]

## Call the Vectorize Image API

In [ ]:
import requests

# Function to vectorize an image
def vectorize_image(image_source, is_url=True):
    # API URL
    url = f"{azure_computer_vision_endpoint}/computervision/retrieval:vectorizeImage?api-version=2024-02-01&model-version=2023-04-15"

    headers = {
        "Ocp-Apim-Subscription-Key": azure_computer_vision_key
    }

    try:
        if is_url:
            # Set headers for URL
            headers["Content-Type"] = "application/json"
            data = {
                "url": image_source
            }
            # Make the request
            response = requests.post(url, headers=headers, json=data)
        else:
            # Read the image file
            with open(image_source, "rb") as image_file:
                image_data = image_file.read()

            # Set headers for image file
            headers["Content-Type"] = "application/octet-stream"
            # Make the request
            response = requests.post(url, headers=headers, data=image_data)

        response.raise_for_status()  # Raise an exception for HTTP errors

        # Return the response
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [ ]:
bus1_result = vectorize_image("../data/customclassificationchallenge/bus/bus1.jpeg", False)
print("Bus 1: ", bus1_result["vector"])

bus2_result = vectorize_image("../data/customclassificationchallenge/bus/bus2.jpeg", False)
print("Bus 2: ", bus2_result["vector"])

japcherry1_result = vectorize_image("../data/customclassification/japanese_cherry/japanese_cherry_1.jpg", False)
print("Japanese Cherry 1: ", japcherry1_result["vector"])

japcherry2_result = vectorize_image("../data/customclassification/japanese_cherry/japanese_cherry_2.jpg", False)
print("Japanese Cherry 2: ", japcherry2_result["vector"])


## Calculate Vector Similarity

In [ ]:
import numpy as np

def cosine_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

In [ ]:
print("Bus 1 vs Bus 2")
print(cosine_similarity(bus1_result["vector"], bus2_result["vector"]))

print("Bus vs Japanese Cherry")
print(cosine_similarity(bus1_result["vector"], japcherry1_result["vector"]))
print(cosine_similarity(bus2_result["vector"], japcherry1_result["vector"]))
print(cosine_similarity(bus1_result["vector"], japcherry2_result["vector"]))
print(cosine_similarity(bus2_result["vector"], japcherry2_result["vector"]))

print("Japanese Cherry 1 vs Japanese Cherry 2")
print(cosine_similarity(japcherry1_result["vector"], japcherry2_result["vector"]))


## Call the Vectorize Text API

In [ ]:
import requests

def vectorize_text(text):
    
    # API URL
    url = f"{azure_computer_vision_endpoint}/computervision/retrieval:vectorizeText?api-version=2024-02-01&model-version=2023-04-15"

    # Set headers
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": azure_computer_vision_key
    }

    # Set the data payload
    data = {
        "text": text
    }

    try:
        # Make the request
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Return the JSON response
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None


In [ ]:
user_input = "bus"
text_vector = vectorize_text(user_input)
print("Text: ", user_input)
print(cosine_similarity(text_vector["vector"], bus1_result["vector"]))
print(cosine_similarity(text_vector["vector"], bus2_result["vector"]))
print(cosine_similarity(text_vector["vector"], japcherry1_result["vector"]))
print(cosine_similarity(text_vector["vector"], japcherry2_result["vector"]))

user_input = "japanese cherry"
print("Text: ", user_input)
text_vector = vectorize_text(user_input)
print(cosine_similarity(text_vector["vector"], bus1_result["vector"]))
print(cosine_similarity(text_vector["vector"], bus2_result["vector"]))
print(cosine_similarity(text_vector["vector"], japcherry1_result["vector"]))
print(cosine_similarity(text_vector["vector"], japcherry2_result["vector"]))
